# Fixed-points of Boolean networks

AEON.py includes methods for (relatively) efficient symbolic computation of all model fixed-points. 
Note that if you are only interested in *some* fixed-points, for very large models, it's probably 
faster to use some kind of SAT/ASP based method. However, if you need to further manipulate the 
set of fixed-points or compute it's cardinality, it might be infeasible to enumerate them all 
using a solver. That's where AEON.py comes in.

> To execute this notebook, you'll need some functionality that is currently available only in 
  a pre-release version of AEON.py. Please make sure you have at least version `0.2.0a4` installed
  (`pip install biodivine_aeon==0.2.0a4`). Once final `0.2.0` becomes generally available later in 2023, 
  you can just use that.
  

To test the feature, we'll use models from the [BBM benchmark](https://github.com/sybila/biodivine-boolean-models). 
Alternatively, you can use any `.aeon`, `.sbml` or `.bnet` model. Just be aware that various authors tend to 
represent model inputs (constant variables with no regulations) differently. So if the results seem abnormal 
for your model, try to see if inputs are correctly recognized (although this shouldn't matter for fixed-points
as much as for other verification problems). BBM already does a decent job at cleaning this up,
but if you use models from some other sources, you're on your own ;)

In [1]:
import subprocess
import os
import time
from biodivine_aeon import *

In [2]:
# Quitely fetch BBM models and unzip them into `models` directory.
subprocess.run(["wget", "-q", "-N", "https://github.com/sybila/biodivine-boolean-models/releases/download/august-2022/edition-2022-aeon.zip"])
subprocess.run(["unzip", "-q", "-o", "-j", "edition-2022-aeon.zip", "-d", "models"]);

In [3]:
# Load all networks (the directory also contains a summary .csv file and metadata.json, so we want to skip that).
# Since the OS can return the files in arbitrary order, we want to sort them to preserve the expected benchmark order.
model_files = [ x for x in os.listdir("models") if x.endswith(".aeon") ]
model_files = sorted(model_files, key=lambda x: int(x.replace(".aeon", "")))
models = [ BooleanNetwork.from_file(f"models/{file}") for file in model_files ]
models[:10]

[BooleanNetwork(variables = 321, parameters = 0, regulations = 533),
 BooleanNetwork(variables = 139, parameters = 0, regulations = 557),
 BooleanNetwork(variables = 20, parameters = 0, regulations = 51),
 BooleanNetwork(variables = 247, parameters = 0, regulations = 1100),
 BooleanNetwork(variables = 28, parameters = 0, regulations = 123),
 BooleanNetwork(variables = 68, parameters = 0, regulations = 103),
 BooleanNetwork(variables = 5, parameters = 0, regulations = 14),
 BooleanNetwork(variables = 28, parameters = 0, regulations = 45),
 BooleanNetwork(variables = 73, parameters = 0, regulations = 114),
 BooleanNetwork(variables = 15, parameters = 0, regulations = 37)]

#### Input inlining (optional)

At this point, all model inputs are represented as variables with constant update functions
(this representation has the best compatibility with other tools). However, this is a bit 
inefficient for BDD representation. We can ask AEON to automatically recognize constant input 
variables and turn them into parameters. Overall, this is probably not going to turn an unsolvable
problem into a solvable one, but can still save you some computation time (and quite a bit of RAM
for larger models).
 
However, keep in mind that AEON refers to such parameter valuations as colors: so the result
won't be just a set of states, but a relation over states and colors. If you don't perform this
transformation, you don't have to worry about colors: your models won't contain any, all information will be part of the network state.

In [4]:
models = [ model.inline_inputs() for model in models ]
models[:10]

[BooleanNetwork(variables = 302, parameters = 19, regulations = 407),
 BooleanNetwork(variables = 130, parameters = 9, regulations = 540),
 BooleanNetwork(variables = 19, parameters = 1, regulations = 48),
 BooleanNetwork(variables = 225, parameters = 22, regulations = 1058),
 BooleanNetwork(variables = 28, parameters = 0, regulations = 123),
 BooleanNetwork(variables = 62, parameters = 6, regulations = 96),
 BooleanNetwork(variables = 5, parameters = 0, regulations = 14),
 BooleanNetwork(variables = 25, parameters = 3, regulations = 41),
 BooleanNetwork(variables = 60, parameters = 13, regulations = 69),
 BooleanNetwork(variables = 13, parameters = 2, regulations = 35)]

We can see that now the networks also contain parameters, but the sum of `parameters` and `variables` 
is the same as for the original model (the number of regulations decreased because influence from 
parameters does not count towards regulations right now). However, the transition systems of these new 
networks should be isomorphic to the old ones, it's only a change in encoding.

 > Note that not all inputs are always safe to inline. While this is largely an edge-case, if the input `I` 
   regulates variable `V` but does not actually appear *in* the update function of `V`, inlining it would
   erase it from the model completely, in which case we don't do it (while this sounds absurd, this actually
   appears in some models).

Finally, we have to build the state-transition graphs for each model (these effectively "manage" the
symbolic encoding; `BooleanNetwork` only represents the model).

In [5]:
STGs = [ SymbolicAsyncGraph(model) for model in models ]

### Solver-based fixed-point detection

First, we show how to use a solver in AEON.py to detect fixed-points. Internally, this uses Z3, 
which is a bit of an overkill for this kind of task but is relatively reliable and easy to integrate. 

 > The solver is baked into AEON.py, so you don't have to install Z3 independently.

 > WARNING: Right now, the result is exported to Python as a list, not an iterator. Internally 
   (in Rust), the result is an iterator, but I didn't have time to export it to Python as a proper
   iterator too. Just keep in mind that all results have to fit into memory in this case. If this
   is a limitation for you (i.e. you want a true iterator), get it touch and I will add it.

In [6]:
for stg in STGs[:20]:
    start = time.time()
    fixed_point_list = FixedPoints.solver_list(stg, limit=10)
    print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Found {len(fixed_point_list)} fixed-points in {round(time.time() - start, 2)}s.")

[302;19] Found 10 fixed-points in 0.05s.
[130;9] Found 10 fixed-points in 0.15s.
[19;1] Found 3 fixed-points in 0.01s.
[225;22] Found 10 fixed-points in 0.37s.
[28;0] Found 0 fixed-points in 0.02s.
[62;6] Found 10 fixed-points in 0.02s.
[5;0] Found 2 fixed-points in 0.01s.
[25;3] Found 10 fixed-points in 0.02s.
[60;13] Found 10 fixed-points in 0.02s.
[13;2] Found 6 fixed-points in 0.01s.
[40;4] Found 10 fixed-points in 0.02s.
[94;7] Found 10 fixed-points in 0.03s.
[32;2] Found 4 fixed-points in 0.01s.
[54;7] Found 10 fixed-points in 0.03s.
[14;2] Found 2 fixed-points in 0.01s.
[104;14] Found 10 fixed-points in 0.02s.
[41;9] Found 10 fixed-points in 0.02s.
[76;28] Found 10 fixed-points in 0.02s.
[71;15] Found 10 fixed-points in 0.02s.
[39;2] Found 0 fixed-points in 0.01s.


Note that the results of this enumeration (members of the list) are still symbolic sets. They just all 
contian a single state and a single color. This is mostly to ensure compatibility with other parts 
of the AEON.py API. This means you can treat them as sets and perform operations like 
`union` or `intersection` on them.

### Symbolic fixed-point detection

If a network has *a lot* of fixed-points, it might not be possible to enumerate them all using a solver. 
That's where a symbolic algorithm can be useful, but keep in mind that the computation can take much 
longer for large networks.

 > The numbers below are from an i7-4790 with 32GB of DDR3. This is an almost 10-year-old CPU, so
   unless you are running this on Raspberry Pi, your results should probably come out similar or better :) 
   Also, for the vast majority of networks, 4-8GB of RAM or even less should be fine.
   
 > If you want to compare to a more recent CPU, `fixed-points-symbolic.csv` contains results for all models
   from a Ryzen 5800X. As you can see, the vast majority of models finished within a few seconds, but there are
   some outliers that need ~1h to complete (these have more than 2^100 fixed-points though).

In [7]:
for stg in STGs[:50]:
    if stg.network().num_parameters() > 50:
        # A large number of parameters typically means a large number of
        # fixed-points and a long computation. For demonstration purposes,
        # we can skip those. 
        print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Skipped.", flush=True)
        continue
    start = time.time()
    fixed_points = FixedPoints.symbolic(stg)
    print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Found {int(fixed_points.cardinality())} fixed-points in {round(time.time() - start, 2)}s.", flush=True)

[302;19] Found 471040 fixed-points in 16.53s.
[130;9] Found 32768 fixed-points in 0.22s.
[19;1] Found 3 fixed-points in 0.0s.
[225;22] Found 3005341696 fixed-points in 2.19s.
[28;0] Found 0 fixed-points in 0.0s.
[62;6] Found 72 fixed-points in 0.03s.
[5;0] Found 2 fixed-points in 0.0s.
[25;3] Found 27 fixed-points in 0.0s.
[60;13] Found 4096 fixed-points in 0.03s.
[13;2] Found 6 fixed-points in 0.0s.
[40;4] Found 16 fixed-points in 0.0s.
[94;7] Found 104 fixed-points in 0.1s.
[32;2] Found 4 fixed-points in 0.0s.
[54;7] Found 172 fixed-points in 0.02s.
[14;2] Found 2 fixed-points in 0.0s.
[104;14] Found 16384 fixed-points in 0.06s.
[41;9] Found 2050 fixed-points in 0.02s.
[76;28] Found 197132288 fixed-points in 0.17s.
[71;15] Found 28672 fixed-points in 0.06s.
[39;2] Found 0 fixed-points in 0.0s.
[14;3] Found 10 fixed-points in 0.0s.
[17;5] Found 58 fixed-points in 0.0s.
[9;1] Found 1 fixed-points in 0.0s.
[16;4] Found 20 fixed-points in 0.0s.
[54;6] Found 82 fixed-points in 0.02s.
[18;

You can also use a `restriction` parameter (this can be any symbolic `ColoredVertexSet`) that restricts the search only to this particular subset of the state space:

In [8]:
stg = STGs[0]
model = stg.network()
[model.get_variable_name(x) for x in model.variables()][:10]

['v_APAF1',
 'v_APAF1_CYCS',
 'v_APAF1gene',
 'v_ATF2',
 'v_Apoptosis',
 'v_Apoptosome',
 'v_BAD',
 'v_BAG4',
 'v_BAG4_TNFRSF1A',
 'v_BAK1']

For example, here we test if the absence of `BAK1` has an impact on the presence of `Apoptosis` 
in model fixed-points, and as it turns out, it does:

In [9]:
symbolic_space = stg.fix_subspace({ "v_BAK1": False, "v_Apoptosis": True })
start = time.time()
fixed_points = FixedPoints.symbolic(stg, restriction=symbolic_space)
print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Found {int(fixed_points.cardinality())} fixed-points in {round(time.time() - start, 2)}s.", flush=True)

symbolic_space = stg.fix_subspace({ "v_BAK1": False, "v_Apoptosis": False })
start = time.time()
fixed_points = FixedPoints.symbolic(stg, restriction=symbolic_space)
print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Found {int(fixed_points.cardinality())} fixed-points in {round(time.time() - start, 2)}s.", flush=True)

[302;19] Found 159520 fixed-points in 8.17s.
[302;19] Found 69856 fixed-points in 2.6s.


Note that if you already have the complete symbolic set of fixed-points, you can get the same result 
just by intersecting it with the `symbolic_space`. But as you can see, if you add enough restrictions,
it might be faster to just compute the two results independently.

#### Projections

In some instances, you only care about the fixed-point states (without the colors; i.e. input valuations), 
or about fixed-point colors (input valuations that admit existence of *some* fixed-point). In these cases,
you can use a projection to only obtain this particular part of the result. In most cases, this is measurably
faster than computing the full result.

In [10]:
for stg in STGs[:50]:
    if stg.network().num_parameters() > 50:
        # A large number of parameters typically means a large number of
        # fixed-points and a long computation. For demonstration purposes,
        # we can skip those. 
        print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}] Skipped.", flush=True)
        continue
    start = time.time()
    fixed_points = FixedPoints.symbolic_vertices(stg)
    fixed_point_colors = FixedPoints.symbolic_colors(stg)
    print(f"[{stg.network().num_vars()};{stg.network().num_parameters()}]", flush=True)
    print(f" > {int(fixed_points.cardinality())}/{int(stg.unit_colored_vertices().vertices().cardinality())} fixed-point vertices (states).")
    print(f" > {int(fixed_point_colors.cardinality())}/{int(stg.unit_colors().cardinality())} colors (input valuations).")
    print(f" > Elapsed: {round(time.time() - start, 2)}s.")

[302;19]
 > 210944/8148143905337944345073782753637512644205873574663745002544561797417525199053346824733589504 fixed-point vertices (states).
 > 458752/524288 colors (input valuations).
 > Elapsed: 16.02s.
[130;9]
 > 7488/1361129467683753853853498429727072845824 fixed-point vertices (states).
 > 32/512 colors (input valuations).
 > Elapsed: 0.37s.
[19;1]
 > 3/524288 fixed-point vertices (states).
 > 2/2 colors (input valuations).
 > Elapsed: 0.0s.
[225;22]
 > 2661504/53919893334301279589334030174039261347274288845081144962207220498432 fixed-point vertices (states).
 > 446720/4194304 colors (input valuations).
 > Elapsed: 2.69s.
[28;0]
 > 0/268435456 fixed-point vertices (states).
 > 0/1 colors (input valuations).
 > Elapsed: 0.0s.
[62;6]
 > 13/4611686018427387904 fixed-point vertices (states).
 > 64/64 colors (input valuations).
 > Elapsed: 0.03s.
[5;0]
 > 2/32 fixed-point vertices (states).
 > 1/1 colors (input valuations).
 > Elapsed: 0.0s.
[25;3]
 > 16/33554432 fixed-point vertices 

As always with projections, keep in mind that the set of fixed points is more complex than just 
the product of the vertex and color projection.

Right now, we can only do projections to vertices (`VertexSet`) and colors (`ColorSet`). However, the underlying Rust code
can actually do projections to any subset of variables or parameters. The main limitation is that
we don't have a type safe way of manipulating such projections yet (e.g. what is the result of a union between a "general" symbolic set
and a projection? Should this even be allowed?), so it is only an "unsafe" experimental feature for now that's 
not available in Python. If you want to use it for something, get in touch :)